In [1]:
 import os
import numpy as np
from keras.models import Sequential # To initialise the nn as a sequence of layers
from keras.layers import Convolution2D # To make the convolution layer for 2D images
from keras.layers import MaxPooling2D # 
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import CSVLogger
from keras.optimizers import RMSprop
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

csv=CSVLogger("8_preprocessing_model.log")
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32,(2,2),input_shape = (224,224,1),strides=2,name='convo1'))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(64,(3,3), activation = 'relu',name='convo2'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# Step 1 - Convolution
classifier.add(Convolution2D(64,(3,3),activation = 'relu',name='convo3'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

# classifier.add(BatchNormalization())
# As GlobalAveragePooling2D take the average of all the dimensions((26, 26, 64) --> (64)), so normalizing at this 
# spot seems misleading.
classifier.add(GlobalAveragePooling2D())
# As our model is still facing the problem so, we need to increase the regulization
classifier.add(Dropout((0.5)))
classifier.add(Dense(20, activation = 'softmax'))

Using TensorFlow backend.


In [2]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convo1 (Conv2D)              (None, 112, 112, 32)      160       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
convo2 (Conv2D)              (None, 110, 110, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
convo3 (Conv2D)              (None, 53, 53, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

In [6]:
adam = Adam(epsilon = 0.01,lr = 0.001)
classifier.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,featurewise_center = True,samplewise_center = True,
                                   featurewise_std_normalization = True,samplewise_std_normalization = True,
                                  width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

curr_path = os.getcwd()
basefolder = os.path.dirname(curr_path)

train_folder = os.path.join(basefolder, "Dataset\\Train")
test_folder = os.path.join(basefolder, "Dataset\\dev")
"""
# Changes for linux
train_folder = os.path.join(basefolder, "Dataset/Train")
test_folder = os.path.join(basefolder, "Dataset/dev")
"""
train_set = train_datagen.flow_from_directory(train_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

test_set = test_datagen.flow_from_directory(test_folder,target_size=(224, 224),batch_size=32,class_mode='categorical',color_mode='grayscale')

Found 2600 images belonging to 20 classes.
Found 200 images belonging to 20 classes.


In [7]:
history = classifier.fit_generator(train_set,steps_per_epoch=2600,epochs=10,validation_data=test_set,validation_steps=200,callbacks=[csv],verbose=2)

Epoch 1/10


C:\Users\dhruv\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\preprocessing\image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\dhruv\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\preprocessing\image.py:514: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


400s - loss: 2.9960 - acc: 0.0452 - val_loss: 3.3831 - val_acc: 0.0503
Epoch 2/10
395s - loss: 2.9960 - acc: 0.0444 - val_loss: 3.3832 - val_acc: 0.0498
Epoch 3/10
395s - loss: 2.9960 - acc: 0.0457 - val_loss: 3.3825 - val_acc: 0.0501
Epoch 4/10
387s - loss: 2.9960 - acc: 0.0442 - val_loss: 3.3846 - val_acc: 0.0496
Epoch 5/10
378s - loss: 2.9960 - acc: 0.0452 - val_loss: 3.3856 - val_acc: 0.0494
Epoch 6/10
381s - loss: 2.9960 - acc: 0.0446 - val_loss: 3.3820 - val_acc: 0.0501
Epoch 7/10
392s - loss: 2.9960 - acc: 0.0463 - val_loss: 3.3858 - val_acc: 0.0496
Epoch 8/10
403s - loss: 2.9960 - acc: 0.0458 - val_loss: 3.3855 - val_acc: 0.0500
Epoch 9/10
405s - loss: 2.9960 - acc: 0.0455 - val_loss: 3.3839 - val_acc: 0.0501
Epoch 10/10
394s - loss: 2.9960 - acc: 0.0457 - val_loss: 3.3845 - val_acc: 0.0501
